<h1>Algoritmo Genético - Trabajo IA 2018</h1>
Integrantes

- Sebastian Gaete
- Elard Koch
- Israel Díaz
- Francisco González

**notas**<br>
representación de individuo:
diccionario de python, el cual posee las siguientes claves,valores:

- **doors**: lista de puertas asociadas
- **ticks**: cantidad de ticks promedio del individuo
- **gen**: generación del individuo

una población de  individuos corresponderá a una lista de diccionarios

In [1]:
netlogo = None

In [2]:
import platform
import pyNetLogo
import random
import numpy as np

netlogo_gui=False #ejecutar netlogo con interfaz grafica (True = si, False = no)
current_gen = 0 #contador de la generación actual
max_gen=50 #total de generaciones
n_doors = 5 #numero de puertas a ubicar
pop_size=20 #tamaño poblacion
n_iterations=5 #cantidad de iteraciones por individuo
n_tournament=3 #cantidad de individuos por torneo
p_mut=0.3 #probabilidad de mutacion

plan_file="office.plan"

population = []
good_doors=[] #lista de coordenadas que pueden albergar una puerta 

- Para iniciar netlogo: **launch_netlogo()**
- Para iniciar netlogo: **kill_netlogo()**

In [3]:

def launch_netlogo():
    global netlogo
    print("Trying to start netlogo, please wait...")
    try:
        sistema=platform.system()
        if(sistema=='Windows'):
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui)
        elif(sistema=='Linux'):
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui,netlogo_home='NetLogo 6.0.4/')
        else:
            netlogo = pyNetLogo.NetLogoLink(gui=netlogo_gui)
        print("Done!")
    except Exception as e:
        print("Ocurrio un problema al levantar netlogo!")
        print(e)

def kill_netlogo():
    global netlogo
    if netlogo!=None:
        try:
            netlogo.kill_workspace()
            netlogo = None
        except Exception as e:
            print("An exception has ocurred while attempíng to kill NetLogo!")
            print(e)
    else:
        print("Please start netlogo first before kill it")

In [4]:
#iniciar netlogo
launch_netlogo()

Trying to start netlogo, please wait...
Done!


In [5]:
netlogo.load_model(r'escape4_v6.nlogo')
netlogo.command('set plan-data "{0}"'.format(plan_file))
netlogo.command("load-plan-file")

In [6]:
#detener netlogo
#kill_netlogo()

<h1>Funciones para algoritmo genético</h1>

In [14]:

#simulate recibe la lista ordenada de coordenadas de puertas, realiza la simulación correspondiente y retorna los ticks que ha demorado
#La distribución de puertas debe estar previamente validada, en caso de que las puertas no sean accesibles, la simulación podría quedar atascada en un while infinito
def simulate(doors):
    str_command = "set door-data {0}".format(str(doors).replace(',','').replace("[","[ ").replace("]"," ]")).replace("(","[").replace(")","]")
    netlogo.command(str_command)
    netlogo.command("show-plan")
    netlogo.command("generate-population")
    while(netlogo.report("count turtles")>0):
        netlogo.command("go")
    return(netlogo.report("ticks"))

def simulate_ind(individual):
    doors = list(map(lambda i:[good_doors[i][0],good_doors[i][1]],individual['doors']))
    return simulate(doors)
    
def get_good_doors(): #Obtener las coordenadas x e y de todas las puertas(posibles) que nos sirven
    x=list(map(lambda i:int(i),netlogo.report("listax")))
    y=list(map(lambda i:int(i),netlogo.report("listay")))
    return list(map(lambda i:(x[i],y[i]),range(len(x))))

def gen_zero(): #crea la generacion 0
    tmp_population=[]
    for i in range(pop_size):
        individual = {
            'doors':[],
            'gen':0,
            'ticks':0
        }
        tmp_population.append(individual)
        for j in range(n_doors):
            random_door_index = random.randint(0,len(good_doors) - 1)
            individual['doors'].append(random_door_index)
    return tmp_population
        

def evaluate_population(pop):
    for individual in pop:
        simulations_ticks=[]
        for i in range(n_iterations):
            ticks = simulate_ind(individual)
            simulations_ticks.append(ticks)
        individual['ticks'] = np.mean(simulations_ticks)
        
def do_tournament(pop,n_players):
    result = []
    tmp_pop = pop.copy()
    if(n_players > len(pop)):
        n_players=len(pop)
    while len(tmp_pop) >= n_players:
        print("#")
#        print("len: {0}".format(len(tmp_pop)))
#        for i in tmp_pop:
#            print(i)
         
        players_selection = []
        while(len(players_selection)<n_players):
            selection = np.random.choice(tmp_pop)
            if(selection not in players_selection):
                players_selection.append(selection)
        #players_selection = list(map( lambda i: np.random.choice(tmp_pop) , range(n_players) ))
        
        players_doors=list(map(lambda p:p['doors'],players_selection))
        
        winner = players_selection[np.argmin([ i['ticks'] for i in players_selection])]
        tmp_pop=list(filter(lambda p:p['doors'] not in players_doors,tmp_pop))
#        print("players_selection")
        
#        for i in players_selection:
#            print(i)
        
#        print("winner")
#        print(winner)
        result.append(winner)
        
    if(len(tmp_pop)>0):
#        print("len: {0}".format(len(tmp_pop)))
#        print("#last")
        
#        for i in tmp_pop:
#            print(i)
        
        winner = tmp_pop[np.argmin([ i['ticks'] for i in tmp_pop])]
        result.append(winner)
#        print("players_selection")
#        for i in tmp_pop:
#            print(i)
#        print("winner")
#        print(winner)
        tmp_pop=list(filter(lambda p:p['doors'] not in players_doors,tmp_pop))
    return result


def do_crossover(pop,gen_number=0):
    new_pop=[]
    punto_corte = random.randint(1,n_doors - 1)
    for i in range(len(pop)//2):
        parent1=pop[i]['doors']
        parent2=pop[len(pop)//2 + i]['doors']
        new_individual = {
            'doors':parent1[:punto_corte] + parent2[punto_corte:],
            'gen':gen_number,
            'ticks':0
        }
        new_pop.append(new_individual)
        """
        print("#")
        print("pto_corte:",end=" ")
        print(punto_corte)
        print("padre1:",end=" ")
        print(parent1)
        print("padre2:",end=" ")
        print(parent2)
        print("hijo:",end=" ")
        print(new_individual)
        """
    return new_pop
        
def do_mutate(pop,prob=p_mut):
    new_pop = pop.copy()
    for individual in new_pop:
        if np.random.random() <= prob:
            random_index = random.randint(0,n_doors - 1)
            new_door = np.random.randint(len(good_doors))
            while(new_door == individual['doors'][random_index]):
                new_door = np.random.randint( len(good_doors))
            individual['doors'][random_index] = new_door
            individual['ticks']=0
    return new_pop
            
        
        

In [10]:
good_doors = get_good_doors() #guardar las puertas posibles en good_doors
len(good_doors)

298

<h1>Inicializar la población</h1>

In [ ]:
#inicialización
population = gen_zero()
evaluate_population(population)
population

<h1>Seleccion</h1>

In [293]:
winners = do_tournament(population,2)
winners


[{'doors': [101, 208, 285, 222, 147], 'gen': 0, 'ticks': 105.4},
 {'doors': [214, 45, 267, 296, 52], 'gen': 0, 'ticks': 115.0},
 {'doors': [192, 197, 256, 255, 46], 'gen': 0, 'ticks': 91.2},
 {'doors': [51, 193, 103, 211, 58], 'gen': 0, 'ticks': 103.2},
 {'doors': [13, 86, 280, 28, 110], 'gen': 0, 'ticks': 70.8},
 {'doors': [243, 232, 198, 290, 122], 'gen': 0, 'ticks': 97.0},
 {'doors': [99, 21, 23, 207, 5], 'gen': 0, 'ticks': 102.4},
 {'doors': [111, 161, 269, 233, 266], 'gen': 0, 'ticks': 97.6},
 {'doors': [242, 28, 206, 213, 73], 'gen': 0, 'ticks': 126.4},
 {'doors': [152, 202, 167, 119, 131], 'gen': 0, 'ticks': 101.6}]

<h1>Cruzamiento</h1>

In [294]:
cross = do_crossover(winners,1)
cross

[{'doors': [101, 208, 285, 290, 122], 'gen': 1, 'ticks': 0},
 {'doors': [214, 45, 267, 207, 5], 'gen': 1, 'ticks': 0},
 {'doors': [192, 197, 256, 233, 266], 'gen': 1, 'ticks': 0},
 {'doors': [51, 193, 103, 213, 73], 'gen': 1, 'ticks': 0},
 {'doors': [13, 86, 280, 119, 131], 'gen': 1, 'ticks': 0}]

<h1>Mutacion</h1>

In [295]:
test_pop = winners + cross
test_pop

[{'doors': [101, 208, 285, 222, 147], 'gen': 0, 'ticks': 105.4},
 {'doors': [214, 45, 267, 296, 52], 'gen': 0, 'ticks': 115.0},
 {'doors': [192, 197, 256, 255, 46], 'gen': 0, 'ticks': 91.2},
 {'doors': [51, 193, 103, 211, 58], 'gen': 0, 'ticks': 103.2},
 {'doors': [13, 86, 280, 28, 110], 'gen': 0, 'ticks': 70.8},
 {'doors': [243, 232, 198, 290, 122], 'gen': 0, 'ticks': 97.0},
 {'doors': [99, 21, 23, 207, 5], 'gen': 0, 'ticks': 102.4},
 {'doors': [111, 161, 269, 233, 266], 'gen': 0, 'ticks': 97.6},
 {'doors': [242, 28, 206, 213, 73], 'gen': 0, 'ticks': 126.4},
 {'doors': [152, 202, 167, 119, 131], 'gen': 0, 'ticks': 101.6},
 {'doors': [101, 208, 285, 290, 122], 'gen': 1, 'ticks': 0},
 {'doors': [214, 45, 267, 207, 5], 'gen': 1, 'ticks': 0},
 {'doors': [192, 197, 256, 233, 266], 'gen': 1, 'ticks': 0},
 {'doors': [51, 193, 103, 213, 73], 'gen': 1, 'ticks': 0},
 {'doors': [13, 86, 280, 119, 131], 'gen': 1, 'ticks': 0}]

In [296]:
do_mutate(test_pop)

[{'doors': [101, 208, 285, 222, 147], 'gen': 0, 'ticks': 105.4},
 {'doors': [214, 45, 267, 296, 52], 'gen': 0, 'ticks': 115.0},
 {'doors': [192, 197, 256, 255, 46], 'gen': 0, 'ticks': 91.2},
 {'doors': [51, 193, 103, 211, 58], 'gen': 0, 'ticks': 103.2},
 {'doors': [13, 86, 280, 28, 110], 'gen': 0, 'ticks': 70.8},
 {'doors': [243, 232, 198, 290, 122], 'gen': 0, 'ticks': 97.0},
 {'doors': [99, 21, 23, 207, 5], 'gen': 0, 'ticks': 102.4},
 {'doors': [111, 161, 269, 233, 266], 'gen': 0, 'ticks': 97.6},
 {'doors': [166, 28, 206, 213, 73], 'gen': 0, 'ticks': 126.4},
 {'doors': [152, 202, 167, 119, 131], 'gen': 0, 'ticks': 101.6},
 {'doors': [101, 158, 285, 290, 122], 'gen': 1, 'ticks': 0},
 {'doors': [214, 45, 267, 207, 5], 'gen': 1, 'ticks': 0},
 {'doors': [192, 197, 256, 233, 266], 'gen': 1, 'ticks': 0},
 {'doors': [51, 193, 103, 213, 269], 'gen': 1, 'ticks': 0},
 {'doors': [13, 86, 280, 119, 131], 'gen': 1, 'ticks': 0}]

In [297]:
population

[{'doors': [214, 45, 267, 296, 52], 'gen': 0, 'ticks': 115.0},
 {'doors': [13, 86, 280, 28, 110], 'gen': 0, 'ticks': 70.8},
 {'doors': [243, 232, 198, 290, 122], 'gen': 0, 'ticks': 97.0},
 {'doors': [138, 254, 117, 52, 90], 'gen': 0, 'ticks': 156.2},
 {'doors': [103, 189, 38, 99, 37], 'gen': 0, 'ticks': 134.0},
 {'doors': [254, 84, 159, 92, 287], 'gen': 0, 'ticks': 108.2},
 {'doors': [152, 202, 167, 119, 131], 'gen': 0, 'ticks': 101.6},
 {'doors': [43, 244, 82, 134, 134], 'gen': 0, 'ticks': 123.6},
 {'doors': [101, 208, 285, 222, 147], 'gen': 0, 'ticks': 105.4},
 {'doors': [192, 197, 256, 255, 46], 'gen': 0, 'ticks': 91.2},
 {'doors': [255, 159, 211, 283, 60], 'gen': 0, 'ticks': 141.6},
 {'doors': [129, 73, 107, 247, 231], 'gen': 0, 'ticks': 165.0},
 {'doors': [141, 207, 264, 44, 254], 'gen': 0, 'ticks': 103.4},
 {'doors': [150, 175, 9, 104, 101], 'gen': 0, 'ticks': 147.0},
 {'doors': [51, 193, 103, 211, 58], 'gen': 0, 'ticks': 103.2},
 {'doors': [166, 28, 206, 213, 73], 'gen': 0, 'tic

In [11]:
kill_netlogo()
launch_netlogo()

Please start netlogo first before kill it
Trying to start netlogo, please wait...
Done!


In [12]:
netlogo.load_model(r'escape4_v6.nlogo')

<h1>Bucle evolutivo</h1>

In [13]:
import time


netlogo.command('set plan-data "{0}"'.format(plan_file))
netlogo.command("load-plan-file")

max_gen = 5
pop_size = 10
n_iterations = 1
good_doors = get_good_doors() #guardar las puertas posibles en good_doors

start_time = time.time()
print("Generating population...")
pop2 = gen_zero()
print("Done!")
#print("|{0:_^5}|".format("Gen"))
while current_gen < max_gen:
    print("{0:#^10}".format("Gen " + str(current_gen)))
    print("Population size: {0}".format(len(pop2)))
    print("Evaluating...",end="")
    
    evaluate_population(list(filter(lambda p:p['ticks'] ==0,pop2)))
        
    print("Done!")
    print("Tournament...",end="")
    winners = do_tournament(pop2,3)
    print("Done!")
    print("Crossing...",end="")
    cross = do_crossover(winners, current_gen + 1)
    print("Done!")
    print("Mutation phase...",end="")
    result = do_mutate(cross+winners)
    print("Done!")
    """
    print("Tournament (again)...",end="")
    pop2 = do_tournament(result + pop2,3)
    print("Done!")
    
    """
    pop2=result + pop2
    print("Gen analysis finished")
    elapsed_time = time.time() - start_time
    print("Elapsed time: {0}".format(elapsed_time))
    current_gen = current_gen + 1
print(pop2)
elapsed_time = time.time() - start_time
print("Total elapsed time: {0}".format(elapsed_time))

kill_netlogo()

Generating population...
Done!
##Gen 0###
Population size: 10
Evaluating...Done!
Tournament...#
winner
{'doors': [134, 292, 222, 151, 136], 'gen': 0, 'ticks': 111.0}
#
winner
{'doors': [181, 240, 108, 202, 292], 'gen': 0, 'ticks': 106.0}
#
winner
{'doors': [185, 276, 132, 228, 215], 'gen': 0, 'ticks': 90.0}
winner
{'doors': [203, 224, 36, 8, 10], 'gen': 0, 'ticks': 90.0}
Done!
Crossing...Done!
Mutation phase...Done!
Gen analysis finished
Elapsed time: 16.501298427581787
##Gen 1###
Population size: 16
Evaluating...Done!
Tournament...#
winner
{'doors': [48, 224, 36, 8, 10], 'gen': 0, 'ticks': 84.0}
#
winner
{'doors': [185, 276, 132, 228, 215], 'gen': 0, 'ticks': 90.0}
#
winner
{'doors': [266, 94, 169, 215, 37], 'gen': 0, 'ticks': 114.0}
#
winner
{'doors': [181, 240, 95, 202, 292], 'gen': 0, 'ticks': 98.0}
Done!
Crossing...Done!
Mutation phase...Done!
Gen analysis finished
Elapsed time: 29.024104833602905
##Gen 2###
Population size: 22
Evaluating...Done!
Tournament...#
winner
{'doors': [1

In [16]:
len(pop2)

10

In [3]:
import matplotlib as mpl
